In [1]:
import requests
import pandas as pd
from datetime import datetime
import ccxt
import sqlite3

In [2]:
ex = 'indodax' 
currency_f = 'idr'
currency_c = 'eth'
pair = currency_c.upper() + '/' +currency_f.upper()
now = datetime.utcnow()


In [3]:
def get_fx(currency_f):
    fx = requests.get('https://apilayer.net/api/live?access_key=a1d6d82a3df7cf7882c9dd2b35146d6e&source=USD&format=1').json()
    return fx['quotes']['USD' + currency_f.upper()]

In [4]:
def get_basic(time):
    now_ts = datetime.timestamp(time)
    fx_name = 'USD/' + currency_f.upper()
    fx_rate = get_fx(currency_f)
    basics = {'utc':[now_ts],
     'exchange':[ex],
    'pair':[pair],
     fx_name:[float(fx_rate)]}
    basics = pd.DataFrame(basics)
    return basics

In [3]:
def get_ob(ex, pair):
    ex_instance = eval('ccxt.' + ex)()
    ob = ex_instance.fetch_order_book(pair)
    bid_px = dict(zip([str(i) + '_bid_px' for i in range(1,6)],[float(ob['bids'][i][0]) for i in range(5)]))
    ask_px = dict(zip([str(i) + '_ask_px' for i in range(1,6)],[float(ob['asks'][i][0]) for i in range(5)]))
    bid_sz = dict(zip([str(i) + '_bid_sz' for i in range(1,6)],[float(ob['bids'][i][1]) for i in range(5)]))
    ask_sz = dict(zip([str(i) + '_ask_sz' for i in range(1,6)],[float(ob['asks'][i][1]) for i in range(5)]))
    elements = {}
    elements.update(bid_px)
    elements.update(ask_px)
    elements.update(bid_sz)
    elements.update(ask_sz)
    df = pd.DataFrame({k: [v] for k, v in elements.items()})
    return df

In [4]:
get_ob(ex, pair)

,1_bid_px,2_bid_px,3_bid_px,4_bid_px,5_bid_px,1_ask_px,2_ask_px,3_ask_px,4_ask_px,5_ask_px,1_bid_sz,2_bid_sz,3_bid_sz,4_bid_sz,5_bid_sz,1_ask_sz,2_ask_sz,3_ask_sz,4_ask_sz,5_ask_sz
0,2978000.0,2972000.0,2971000.0,2970000.0,2969000.0,2979000.0,2980000.0,2981000.0,2982000.0,2983000.0,0.203742,5.650467,12.5227,12.350416,52.658856,1.04858,1.144764,0.186939,0.571078,15.895624


In [6]:
basics = get_basic(now)

ob = get_ob(ex, pair)

df = pd.concat([basics,ob],axis =1)



In [30]:
# connect to sql
conn = sqlite3.connect('mysqlite.db')
table_name = "ob_" + ex 

In [ ]:
# append
df.to_sql(table_name, conn, if_exists = 'append')

In [31]:
# check 
pd.read_sql_query("SELECT * from {}".format(table_name),conn).tail()

,index,utc,exchange,pair,USD/IDR,1_bid_px,2_bid_px,3_bid_px,4_bid_px,5_bid_px,...,1_bid_sz,2_bid_sz,3_bid_sz,4_bid_sz,5_bid_sz,1_ask_sz,2_ask_sz,3_ask_sz,4_ask_sz,5_ask_sz
4,0,1.587799e+09,indodax,ETH/IDR,15587.9,2976000.0,2975000.0,2973000.0,2972000.0,2970000.0,...,3.864359,0.060008,12.210755,0.046999,0.875421,25.483135,25.365016,0.033278,15.789677,0.342311
5,0,1.587799e+09,indodax,ETH/IDR,15587.9,2976000.0,2975000.0,2972000.0,2970000.0,2968000.0,...,3.864359,0.060008,0.046999,0.875421,0.060023,13.483135,25.365016,0.033278,27.789677,0.342311
6,0,1.587799e+09,indodax,ETH/IDR,15587.9,2975000.0,2972000.0,2970000.0,2968000.0,2967000.0,...,0.060008,0.046999,0.875421,0.060023,0.035804,46.633092,38.483135,0.774417,12.033278,15.789677
7,0,1.587799e+09,indodax,ETH/IDR,15587.9,2971000.0,2970000.0,2968000.0,2967000.0,2966000.0,...,34.246983,0.875421,0.060023,0.035804,0.046997,11.457911,5.659314,0.080435,53.019111,1.017211
8,0,1.587800e+09,indodax,ETH/IDR,15587.9,2971000.0,2970000.0,2968000.0,2967000.0,2966000.0,...,26.169323,0.875421,0.060023,0.035804,0.046997,24.935185,11.461100,6.506485,1.017211,13.401409


In [32]:
conn.close()